In [1]:
#upload some libraries
import os
import pandas.testing as tm
import h5py
import numpy as np
import pandas as pd
import math
import sys
import joblib
import subprocess
import statsmodels.api as sm
from scipy import stats

! pip install --upgrade tables
import tables

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# set paths
basedir = '/data/songy4/twas'
datadir = f'{basedir}/data_folder'

Get Pheno (case vs control) data combined with expression data to select case only
##this file should include FID and IID columns at the beginning

In [3]:
#open 10PCs data file to extract pheno information

pheno_df = pd.read_csv(f"{datadir}/baseline_VST_proteincoding_omics_OG_PPMI_AND_PDBP_ALLTRANSCRIPTS_10PCs_loadings.csv", engine='c')
pheno_df.head()

,ID,PHENO,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,PD-PDAA503EF5,1,-15.766615,-7.416137,14.160851,-3.651206,5.231013,-11.412563,0.777266,-2.475148,-1.861705,1.734574
1,PD-PDAA955TCY,0,-1.364502,34.703132,-6.236409,-5.276376,11.094428,-11.496715,5.663455,9.509184,1.767736,18.216882
2,PD-PDAB074CYQ,1,-17.717657,-10.835780,15.286010,-2.126118,-9.560267,2.662986,-2.867685,-6.341811,-4.522803,-0.449737
3,PD-PDAB411CTU,0,-12.303747,-4.720172,11.457554,-3.323035,-11.942817,-0.523552,-10.005249,-2.163355,-1.913181,3.563834
4,PD-PDAB549YWB,1,-14.272500,-4.349387,11.166659,-7.865786,-5.158357,-0.005893,-10.618556,-5.478464,-3.032065,1.662697


In [4]:
#count how manyID -- total number, both case and control
print(pheno_df.shape[0])

1713


In [5]:
#drop PCs from pheno_df
pheno_id = pheno_df.copy()
pheno_id.drop(pheno_id.iloc[:,2:12], axis=1, inplace=True)
pheno_id.head()

,ID,PHENO
0,PD-PDAA503EF5,1
1,PD-PDAA955TCY,0
2,PD-PDAB074CYQ,1
3,PD-PDAB411CTU,0
4,PD-PDAB549YWB,1


In [23]:
#drop pheno from pheno_df
pheno = pheno_df.drop(['PHENO'], axis=1)

#add _exp behind pheno_df
pheno.columns = [str(col) + '_exp' for col in pheno.columns]

pheno = pheno.rename(columns={'ID_exp':'ID'})

pheno.head()

,ID,PC1_exp,PC2_exp,PC3_exp,PC4_exp,PC5_exp,PC6_exp,PC7_exp,PC8_exp,PC9_exp,PC10_exp
0,PD-PDAA503EF5,-15.766615,-7.416137,14.160851,-3.651206,5.231013,-11.412563,0.777266,-2.475148,-1.861705,1.734574
1,PD-PDAA955TCY,-1.364502,34.703132,-6.236409,-5.276376,11.094428,-11.496715,5.663455,9.509184,1.767736,18.216882
2,PD-PDAB074CYQ,-17.717657,-10.835780,15.286010,-2.126118,-9.560267,2.662986,-2.867685,-6.341811,-4.522803,-0.449737
3,PD-PDAB411CTU,-12.303747,-4.720172,11.457554,-3.323035,-11.942817,-0.523552,-10.005249,-2.163355,-1.913181,3.563834
4,PD-PDAB549YWB,-14.272500,-4.349387,11.166659,-7.865786,-5.158357,-0.005893,-10.618556,-5.478464,-3.032065,1.662697


In [6]:
#open normalized expression file PDBP_and_OG_PPMI_Combined_Clinical_Omics.csv

expre_df = pd.read_csv(f"{datadir}/PDBP_and_OG_PPMI_Transcriptomics_ALLTRANSCRIPTS_diffExp_ProteinCoding_ADJUSTED10PCs.csv", engine='c')

#count how manyID -- total number, both case and control
print(expre_df.shape[0])

1713


In [7]:
#add PHENO column on omics_df by matching participant_id_x with ID 
expression = expre_df.merge(pheno_id, on='ID', how='left')

#leave only PHENO=1 to only include only cases, not controls
expression_case = expression.loc[expression['PHENO'] == 1.0]

#drop Pheno column
expression_case = expression_case.drop(['PHENO'], axis=1)

#grab ID column and duplicate into two columns to compare with .fam file FID IID
expression_case[['IID']] = expression_case[['ID']]
expression_case = expression_case.rename(columns={'ID':'FID'})
expression_case.head()

,FID,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000285937,ENSG00000285938,ENSG00000285946,ENSG00000285947,ENSG00000285950,ENSG00000285953,ENSG00000285975,ENSG00000285976,ENSG00000285982,IID
0,PD-PDAA503EF5,1.412056,-0.523395,0.153490,-1.195026,-0.094739,0.278269,-1.606817,-1.231579,-0.471478,...,0.085944,-0.274379,-0.065663,-0.126903,0.033090,-0.499505,0.059806,-1.298476,0.069994,PD-PDAA503EF5
2,PD-PDAB074CYQ,-0.856754,0.111547,0.741327,0.725518,1.231583,0.593704,-1.066773,0.544353,-1.111941,...,0.051746,-0.786312,-0.003344,-0.166982,-0.059010,-0.499505,0.090575,-2.259028,0.097690,PD-PDAB074CYQ
4,PD-PDAB549YWB,1.118373,0.096341,0.427937,-1.107550,0.040538,-1.836332,-0.428239,1.349854,-1.249725,...,-0.013945,-0.382129,-0.018791,-0.152562,0.084933,-0.499505,0.063530,-0.738222,0.053944,PD-PDAB549YWB
5,PD-PDAB762PA3,-0.816394,-0.318743,-0.632429,-1.268311,0.100997,0.499943,-0.195359,1.295541,0.593312,...,0.096710,0.624501,0.031793,-0.143065,0.120146,-0.499505,-0.050857,-0.730364,-0.024675,PD-PDAB762PA3
9,PD-PDAD926WRZ,-0.248133,0.066066,-1.023469,-2.544938,-0.100176,0.210245,0.317189,-0.789634,-0.752465,...,-0.037379,0.457722,-0.070049,-0.139220,0.148462,0.037965,-0.020618,-1.623932,0.020244,PD-PDAD926WRZ


In [8]:
#function to move columns
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

#rearrange Gene_Symbol, Chr, Start, Stop columns right nextg to index column 
expression_case = movecol(expression_case, cols_to_move=['IID'], ref_col='FID', place='After')

expression_case = expression_case.reset_index(drop=True)
print('shape of case only expression data:', expression_case.shape)
expression_case.head()

#case only expression dataset is ready to use

shape of case only expression data: (1138, 19687)


,FID,IID,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,...,ENSG00000285920,ENSG00000285937,ENSG00000285938,ENSG00000285946,ENSG00000285947,ENSG00000285950,ENSG00000285953,ENSG00000285975,ENSG00000285976,ENSG00000285982
0,PD-PDAA503EF5,PD-PDAA503EF5,1.412056,-0.523395,0.153490,-1.195026,-0.094739,0.278269,-1.606817,-1.231579,...,-0.212292,0.085944,-0.274379,-0.065663,-0.126903,0.033090,-0.499505,0.059806,-1.298476,0.069994
1,PD-PDAB074CYQ,PD-PDAB074CYQ,-0.856754,0.111547,0.741327,0.725518,1.231583,0.593704,-1.066773,0.544353,...,-0.364937,0.051746,-0.786312,-0.003344,-0.166982,-0.059010,-0.499505,0.090575,-2.259028,0.097690
2,PD-PDAB549YWB,PD-PDAB549YWB,1.118373,0.096341,0.427937,-1.107550,0.040538,-1.836332,-0.428239,1.349854,...,-0.305483,-0.013945,-0.382129,-0.018791,-0.152562,0.084933,-0.499505,0.063530,-0.738222,0.053944
3,PD-PDAB762PA3,PD-PDAB762PA3,-0.816394,-0.318743,-0.632429,-1.268311,0.100997,0.499943,-0.195359,1.295541,...,-0.075632,0.096710,0.624501,0.031793,-0.143065,0.120146,-0.499505,-0.050857,-0.730364,-0.024675
4,PD-PDAD926WRZ,PD-PDAD926WRZ,-0.248133,0.066066,-1.023469,-2.544938,-0.100176,0.210245,0.317189,-0.789634,...,-0.149070,-0.037379,0.457722,-0.070049,-0.139220,0.148462,0.037965,-0.020618,-1.623932,0.020244


In [9]:
#Example_phenotype_file.txt from github
example = 'https://raw.githubusercontent.com/opain/Calculating-FUSION-TWAS-weights-pipeline/master/Example_phenotype_file.txt'
df_example = pd.read_csv(example, sep='\s+')
df_example.head()

,FID,IID,ENSG00000223972,ENSG00000227232,ENSG00000237613,ENSG00000238009,ENSG00000239945,ENSG00000233750,ENSG00000268903,ENSG00000241860
0,380928,380928,0.221062,-1.095390,-1.396011,-0.673701,0.562634,-0.340576,0.674828,-0.619243
1,198723,198723,-2.083293,0.581032,2.296874,-2.466542,0.197751,0.894726,1.523388,0.032913
2,109803,109803,-1.420241,1.381628,-0.035638,-1.426041,1.256708,-0.041527,0.470557,0.738527
3,1987391,1987391,-0.203393,-0.052965,-2.009493,-0.743823,0.337753,-0.270654,-1.097861,0.042117
4,123414,123414,-2.099258,0.065376,0.069495,-0.238382,-2.003622,0.191987,-0.049066,-0.999551


In [9]:
#extract genetic PCs from expression data 
!plink --bfile /{datadir}/AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI --pca 10 --out /{datadir}/PCA

PLINK v1.90b4.4 64-bit (21 May 2017)           www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to //data/songy4/twas/data_folder/PCA.log.
Options in effect:
  --bfile //data/songy4/twas/data_folder/AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI
  --out //data/songy4/twas/data_folder/PCA
  --pca 10

1547809 MB RAM detected; reserving 773904 MB for main workspace.
23858199 variants loaded from .bim file.
1997 people (1191 males, 806 females) loaded from .fam.
1844 phenotype values loaded from .fam.
Using up to 143 threads (change this with --threads).
Before main variant filters, 1997 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
//data/songy4/twas/data_folder/PCA.hh ); many commands treat these as missing.


In [14]:
#read genetic PCs
pcs_df = pd.read_csv(f"{datadir}/PCA.eigenvec", sep=' ', usecols=[1,2,3,4,5,6,7,8,9,10,11], names=['ID','PC1_gen','PC2_gen','PC3_gen','PC4_gen','PC5_gen','PC6_gen','PC7_gen','PC8_gen','PC9_gen','PC10_gen'])
pcs_df

,ID,PC1_gen,PC2_gen,PC3_gen,PC4_gen,PC5_gen,PC6_gen,PC7_gen,PC8_gen,PC9_gen,PC10_gen
0,PD-PDAA503EF5,0.002470,-0.003959,-0.001653,0.000744,0.000853,-0.000336,0.000615,0.000049,-0.000596,0.000886
1,PD-PDAA852XL7,0.003624,-0.006965,-0.004234,-0.000906,0.001070,0.000792,0.001020,-0.001415,0.000061,0.000866
2,PD-PDAA955TCY,0.003665,-0.005951,-0.003591,0.000728,0.000924,-0.000524,-0.001126,-0.001011,0.000473,-0.000057
3,PD-PDAB074CYQ,0.003028,-0.005959,-0.004660,-0.001468,-0.001162,-0.000704,-0.005008,0.000753,0.000317,0.000158
4,PD-PDAB411CTU,0.002395,-0.004831,-0.002924,0.000033,0.000973,0.000205,0.000754,-0.000845,0.000794,0.002811
...,...,...,...,...,...,...,...,...,...,...,...
1992,PP-4126,0.003207,-0.004924,-0.003516,0.000243,0.000029,0.000137,-0.000103,-0.001389,-0.000686,0.000260
1993,PP-4127,0.000741,-0.000308,-0.000660,0.001759,0.000649,-0.000244,0.001068,-0.001482,0.000012,-0.000034
1994,PP-4135,-0.021394,0.058516,0.051938,0.014075,0.000885,0.002686,0.012527,-0.005588,0.007664,0.001302
1995,PP-4136,0.003769,-0.006528,-0.003886,-0.000414,0.001516,-0.000477,-0.002527,0.000252,-0.001974,0.000660


Create fid_df.txt file to remove rows in .bim file if FID is missing

In [15]:
#grab FID and IID to compare with .fam file FID IID
id_df= expression_case[['FID', 'IID']].copy()
print('shape of case only id_df:', id_df.shape)
id_df.head()

shape of case only id_df: (1138, 2)


,FID,IID
0,PD-PDAA503EF5,PD-PDAA503EF5
1,PD-PDAB074CYQ,PD-PDAB074CYQ
2,PD-PDAB549YWB,PD-PDAB549YWB
3,PD-PDAB762PA3,PD-PDAB762PA3
4,PD-PDAD926WRZ,PD-PDAD926WRZ


In [54]:
#save id_df as txt  -- run only once for download
id_df.to_csv(r'./data_folder/fid_df.txt', sep='\t' ,index=False)
#id_df.to_csv(r'./data_folder/fid_df.csv' ,index=False)

Plink file preprocess

In [61]:
#look into genotype data folder to grab PLINK files
! ls /data/LNG/makariousmb/Projects/GenoMLxAMP/clean_wgs/PDBP_AND_OG_PPMI

AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI.bed
AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI.bim
AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI.fam
AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI.hh
AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI.log
PCs


In [62]:
#look at the fam file (fid, iid, mid, pid, gender, affection status)
! head /data/LNG/makariousmb/Projects/GenoMLxAMP/clean_wgs/PDBP_AND_OG_PPMI/AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI.fam

PD-PDAA503EF5 PD-PDAA503EF5 0 0 2 2
PD-PDAA852XL7 PD-PDAA852XL7 0 0 1 -9
PD-PDAA955TCY PD-PDAA955TCY 0 0 2 1
PD-PDAB074CYQ PD-PDAB074CYQ 0 0 1 2
PD-PDAB411CTU PD-PDAB411CTU 0 0 1 1
PD-PDAB549YWB PD-PDAB549YWB 0 0 1 2
PD-PDAB597LKL PD-PDAB597LKL 0 0 2 -9
PD-PDAB729HWD PD-PDAB729HWD 0 0 1 2
PD-PDAB762PA3 PD-PDAB762PA3 0 0 2 2
PD-PDAB803VAR PD-PDAB803VAR 0 0 1 1


In [11]:
#copy bed bim fam files into data folder - run only once for download
#these plink files are genotic data (genotype)
! cp -R /data/LNG/makariousmb/Projects/GenoMLxAMP/clean_wgs/PDBP_AND_OG_PPMI/AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI.fam /{datadir}/
! cp -R /data/LNG/makariousmb/Projects/GenoMLxAMP/clean_wgs/PDBP_AND_OG_PPMI/AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI.bim /{datadir}/
! cp -R /data/LNG/makariousmb/Projects/GenoMLxAMP/clean_wgs/PDBP_AND_OG_PPMI/AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI.bed /{datadir}/

In [69]:
#check number of IDs in original .fam file
!cat {datadir}/AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI.fam | wc -l

1997


In [72]:
#check number of IDs in original .fam file
!cat {datadir}/AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI.bim | wc -l

#look at the bim file (variance, chromosome and position, allele1 and allele2)
! head {datadir}/AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI.bim 

23858199
1	rs145427775	0	10291	T	C
1	rs55998931	0	10492	T	C
1	rs199896944	0	13504	A	G
1	rs199856693	0	14933	A	G
1	rs201855936	0	14948	A	G
1	rs71252251	0	14976	A	G
1	rs201045431	0	15029	A	G
1	rs368345873	0	15208	A	G
1	rs374029747	0	15774	A	G
1	rs201330479	0	16792	A	G


In [30]:
#filter sample id using fid_df.txt to reduce rows that are not in fid_df.txt
##plink --bfile <genotype file> --keep <id file> --make-bed --out <outfile>
!plink --bfile /{datadir}/AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI --keep /{datadir}/fid_df.txt  --make-bed --out /{datadir}/qc_genotypes_twas

PLINK v1.90b4.4 64-bit (21 May 2017)           www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /data/songy4/TWAS/data/id_comparison/qc_genotypes_twas.log.
Options in effect:
  --bfile /data/LNG/makariousmb/Projects/GenoMLxAMP/clean_wgs/PDBP_AND_OG_PPMI/AMP_Euro_sampleQC_variantQC_FINAL_PDBP_AND_OG_PPMI
  --keep /data/songy4/TWAS/data/id_df.txt
  --make-bed
  --out /data/songy4/TWAS/data/id_comparison/qc_genotypes_twas

386449 MB RAM detected; reserving 193224 MB for main workspace.
23858199 variants loaded from .bim file.
1997 people (1191 males, 806 females) loaded from .fam.
1844 phenotype values loaded from .fam.
--keep: 1138 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1138 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646

In [58]:
#check the number of IDs in .fam file to make sure it is same numbers as number of rows in exp_df
!cat /{datadir}/qc_genotypes_twas.fam | wc -l

1138


In [73]:
#check number of IDs in original .fam file
!cat {datadir}/qc_genotypes_twas.bim | wc -l

#look at the bim file (variance, chromosome and position, allele1 and allele2)
! head {datadir}/qc_genotypes_twas.bim 

23858199
1	rs145427775	0	10291	T	C
1	rs55998931	0	10492	T	C
1	rs199896944	0	13504	A	G
1	rs199856693	0	14933	A	G
1	rs201855936	0	14948	A	G
1	rs71252251	0	14976	A	G
1	rs201045431	0	15029	A	G
1	rs368345873	0	15208	A	G
1	rs374029747	0	15774	A	G
1	rs201330479	0	16792	A	G


Covariate data preprocess

In [32]:
#open covariate file clinicodemographic_3nov2020.csv
covariate = pd.read_csv(f"{datadir}/clinicodemographic_3nov2020.csv", index_col=False )

print('shape of covariate data:', covariate.shape)

shape of covariate data: (3941, 7)


In [33]:
#drop cov_df ID not in exp_case ID
covariate = covariate[covariate['ID'].isin(expression_case['FID'])]

print('shape of covariate data:', covariate.shape)

#grab ID column and duplicate into two columns to compare with .fam file FID IID
covariate.loc[:,'IID'] = covariate['ID'].values

shape of covariate data: (1138, 7)


In [34]:
#merge pheno and pcs_df into covariate_df
covar_df = covariate.merge(pheno, on='ID', how='left')
covar_df = covar_df.merge(pcs_df, on='ID', how='left')

covar_df.head()

,ID,AGE,MALE,EDUCATION,FAMILY_HISTORY,UPSIT,InfAJ,IID,PC1_exp,PC2_exp,...,PC1_gen,PC2_gen,PC3_gen,PC4_gen,PC5_gen,PC6_gen,PC7_gen,PC8_gen,PC9_gen,PC10_gen
0,PD-PDKG339XM0,56,0,2.0,1.0,17.0,0,PD-PDKG339XM0,10.830733,26.811962,...,0.002876,-0.007443,-0.005085,-0.000187,0.000608,-0.000787,-0.002445,-0.000240,-0.001125,0.000828
1,PD-PDRM726FKG,63,1,2.0,0.0,18.0,0,PD-PDRM726FKG,-6.842610,10.755356,...,0.003366,-0.007768,-0.005870,-0.000552,0.001536,0.000039,-0.000336,0.001626,0.001068,0.001081
2,PD-PDEM487YZD,51,1,1.0,0.0,28.0,0,PD-PDEM487YZD,-2.524943,29.407656,...,0.003569,-0.007966,-0.005733,-0.001055,0.000629,0.000100,0.000309,0.000120,-0.000491,0.001775
3,PD-PDVW658HP7,70,1,2.0,1.0,28.0,0,PD-PDVW658HP7,-5.414782,15.479347,...,0.005395,-0.010209,-0.007062,-0.002003,0.001324,0.001092,-0.000483,0.000103,-0.001346,0.000829
4,PD-PDVJ880MZU,66,1,2.0,0.0,13.0,0,PD-PDVJ880MZU,85.503625,-26.433746,...,0.001514,-0.003853,-0.001543,0.000162,0.000673,-0.000692,-0.000884,-0.000402,0.000285,0.000432


In [35]:
#rename ID to FID
covariate = covar_df.rename(columns={'ID':'FID'})

#rearrange Gene_Symbol, Chr, Start, Stop columns right nextg to index column 
covariate = movecol(covariate, cols_to_move=['IID'], ref_col='FID', place='After')

covariate = covariate.reset_index(drop=True)
print('shape of covariate data:', covariate.shape)

covariate.head()

shape of covariate data: (1138, 28)


,FID,IID,AGE,MALE,EDUCATION,FAMILY_HISTORY,UPSIT,InfAJ,PC1_exp,PC2_exp,...,PC1_gen,PC2_gen,PC3_gen,PC4_gen,PC5_gen,PC6_gen,PC7_gen,PC8_gen,PC9_gen,PC10_gen
0,PD-PDKG339XM0,PD-PDKG339XM0,56,0,2.0,1.0,17.0,0,10.830733,26.811962,...,0.002876,-0.007443,-0.005085,-0.000187,0.000608,-0.000787,-0.002445,-0.000240,-0.001125,0.000828
1,PD-PDRM726FKG,PD-PDRM726FKG,63,1,2.0,0.0,18.0,0,-6.842610,10.755356,...,0.003366,-0.007768,-0.005870,-0.000552,0.001536,0.000039,-0.000336,0.001626,0.001068,0.001081
2,PD-PDEM487YZD,PD-PDEM487YZD,51,1,1.0,0.0,28.0,0,-2.524943,29.407656,...,0.003569,-0.007966,-0.005733,-0.001055,0.000629,0.000100,0.000309,0.000120,-0.000491,0.001775
3,PD-PDVW658HP7,PD-PDVW658HP7,70,1,2.0,1.0,28.0,0,-5.414782,15.479347,...,0.005395,-0.010209,-0.007062,-0.002003,0.001324,0.001092,-0.000483,0.000103,-0.001346,0.000829
4,PD-PDVJ880MZU,PD-PDVJ880MZU,66,1,2.0,0.0,13.0,0,85.503625,-26.433746,...,0.001514,-0.003853,-0.001543,0.000162,0.000673,-0.000692,-0.000884,-0.000402,0.000285,0.000432


In [36]:
#check unique values in each column
for i in covariate.columns[2:8]:
    print('Unique value of ' + i + ':', covariate[i].unique())

#check nan numbers in each column 
print('null value in each column: \n', covariate.isna().sum())

#check data type in all column
print('data types in each column: \n',covariate.dtypes)

Unique value of AGE: [56 63 51 70 66 41 71 78 35 67 43 58 77 72 62 53 69 65 52 50 75 60 68 61
 59 80 86 73 81 74 48 46 64 55 54 57 87 49 76 79 45 85 42 82 84 34 38 44
 37 47 83 39 40]
Unique value of MALE: [0 1]
Unique value of EDUCATION: [2. 1. 0.]
Unique value of FAMILY_HISTORY: [ 1.  0. nan]
Unique value of UPSIT: [17. 18. 28. 13. 23. 32. 29. 21.  6.  9. 31. 15. 34. 14. 22. nan 25. 40.
 16. 33. 20. 19. 37. 10. 11. 12. 26.  8. 36. 35. 24. 30.  7.  4. 27.  1.
 39. 38.  5.]
Unique value of InfAJ: [0 1]
null value in each column: 
 FID                0
IID                0
AGE                0
MALE               0
EDUCATION          0
FAMILY_HISTORY     4
UPSIT             56
InfAJ              0
PC1_exp            0
PC2_exp            0
PC3_exp            0
PC4_exp            0
PC5_exp            0
PC6_exp            0
PC7_exp            0
PC8_exp            0
PC9_exp            0
PC10_exp           0
PC1_gen            0
PC2_gen            0
PC3_gen            0
PC4_gen            0
P

In [37]:
#replace NaN withmean for nonbinary and median for binary column
covariate['FAMILY_HISTORY'] = covariate['FAMILY_HISTORY'].fillna(covariate['FAMILY_HISTORY'].median())
covariate['UPSIT'] = covariate['UPSIT'].fillna(covariate['UPSIT'].mean())

#check nan numbers in each column 
print('null value in each column: \n', covariate.isna().sum())

null value in each column: 
 FID               0
IID               0
AGE               0
MALE              0
EDUCATION         0
FAMILY_HISTORY    0
UPSIT             0
InfAJ             0
PC1_exp           0
PC2_exp           0
PC3_exp           0
PC4_exp           0
PC5_exp           0
PC6_exp           0
PC7_exp           0
PC8_exp           0
PC9_exp           0
PC10_exp          0
PC1_gen           0
PC2_gen           0
PC3_gen           0
PC4_gen           0
PC5_gen           0
PC6_gen           0
PC7_gen           0
PC8_gen           0
PC9_gen           0
PC10_gen          0
dtype: int64


In [38]:
#change float to int in column EDUCATION, FAMILY_HISTORY, and UPSIT

##col=(covariate.dtypes=='float64')
col=covariate.columns[4:7]
covariate.loc[:,col]=covariate.loc[:,col].astype(np.int64)

In [39]:
#normalize covariate values for AGE, UPSIT, EDUCATION columns with min_max_scaling

# apply the min-max scaling in Pandas using the .min() and .max() methods
def min_max_scaling(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.iloc[:, [2, 4, 6]]:
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min()) 
        
    return df_norm
    
# call the min_max_scaling function
covariate_scale = min_max_scaling(covariate)
print('shape of covariate data:', covariate_scale.shape)
covariate_scale.head()

shape of covariate data: (1138, 28)


,FID,IID,AGE,MALE,EDUCATION,FAMILY_HISTORY,UPSIT,InfAJ,PC1_exp,PC2_exp,...,PC1_gen,PC2_gen,PC3_gen,PC4_gen,PC5_gen,PC6_gen,PC7_gen,PC8_gen,PC9_gen,PC10_gen
0,PD-PDKG339XM0,PD-PDKG339XM0,0.415094,0,1.0,1,0.410256,0,10.830733,26.811962,...,0.002876,-0.007443,-0.005085,-0.000187,0.000608,-0.000787,-0.002445,-0.000240,-0.001125,0.000828
1,PD-PDRM726FKG,PD-PDRM726FKG,0.547170,1,1.0,0,0.435897,0,-6.842610,10.755356,...,0.003366,-0.007768,-0.005870,-0.000552,0.001536,0.000039,-0.000336,0.001626,0.001068,0.001081
2,PD-PDEM487YZD,PD-PDEM487YZD,0.320755,1,0.5,0,0.692308,0,-2.524943,29.407656,...,0.003569,-0.007966,-0.005733,-0.001055,0.000629,0.000100,0.000309,0.000120,-0.000491,0.001775
3,PD-PDVW658HP7,PD-PDVW658HP7,0.679245,1,1.0,1,0.692308,0,-5.414782,15.479347,...,0.005395,-0.010209,-0.007062,-0.002003,0.001324,0.001092,-0.000483,0.000103,-0.001346,0.000829
4,PD-PDVJ880MZU,PD-PDVJ880MZU,0.603774,1,1.0,0,0.307692,0,85.503625,-26.433746,...,0.001514,-0.003853,-0.001543,0.000162,0.000673,-0.000692,-0.000884,-0.000402,0.000285,0.000432


In [41]:
#save covariate as txt -- run only once for download
covariate_scale.to_csv(r'./data_folder/covariates.txt', sep='\t' ,index=False)

Coordinate data preprocess

In [42]:
#open gene_chr_start_stop.tab with column names -- this has ch37 chr and position info
##the file "gene_chr_start_stop.tab" is in GP2_WG/working groups/CD_DAWG/scratch/example_TWAS
chr_pos_df = pd.read_csv(f"{datadir}/gene_chr_start_stop.tab", sep='\t', header=None, index_col=False, names=('ID', 'X.Chr', 'start', 'end'))

#rearrange the columns 
chr_pos_df = chr_pos_df[['X.Chr','start','end','ID']]

print('shape of chromosom position data:', chr_pos_df.shape)
chr_pos_df.head()

shape of chromosom position data: (57905, 4)


,X.Chr,start,end,ID
0,1,11869,14412,ENSG00000223972
1,1,14363,29806,ENSG00000227232
2,1,29554,31109,ENSG00000243485
3,1,34554,36081,ENSG00000237613
4,1,52473,54936,ENSG00000268020


In [40]:
#checke chromosome inputs in chr_pos_df
unique_chromosom = chr_pos_df['X.Chr'].unique()
print('unique chromosom:', unique_chromosom)

#checke ID inputs in chr_pos_df -- there is no overlap in ID
nunique_chromosom = chr_pos_df['ID'].nunique()
print('number of unique chromosom:', nunique_chromosom)

unique chromosom: ['1' '2' '3' '4' '5' '6' '7' 'X' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '20' '19' 'Y' '22' '21' 'MT' 'GL000192.1' 'GL000225.1'
 'GL000194.1' 'GL000193.1' 'GL000222.1' 'GL000212.1' 'GL000195.1'
 'GL000223.1' 'GL000224.1' 'GL000219.1' 'GL000205.1' 'GL000215.1'
 'GL000216.1' 'GL000199.1' 'GL000211.1' 'GL000213.1' 'GL000220.1'
 'GL000218.1' 'GL000209.1' 'GL000221.1' 'GL000228.1' 'GL000191.1'
 'GL000204.1' 'GL000233.1' 'GL000237.1' 'GL000230.1' 'GL000242.1'
 'GL000243.1' 'GL000241.1' 'GL000236.1' 'GL000240.1' 'GL000196.1'
 'GL000247.1' 'GL000201.1' 'GL000231.1' 'GL000229.1']
number of unique chromosom: 57905


In [55]:
#make exp_case column names as list without FID and IID
##this will be all ENSG transcripts numbers in our expression data
trans = expression_case.columns[2:19688].tolist()
len(trans)

19685

In [56]:
#make trans list as a dataframe
fid = pd.DataFrame(trans, columns=["ID"])
print("total number of columns in expression data:", fid.shape[0])

#check how many ENSG are missing from FID that are not in ID
mis_coord = fid[~fid['ID'].isin(chr_pos_df['ID'])]
print("number of columns not in expression data:", mis_coord.shape[0])

total number of columns in expression data: 19685
number of columns not in expression data: 602


In [57]:
#drop rows of chr_pos_df where chr_pos_df ID is not match with exp_case column name list
coord = chr_pos_df[chr_pos_df['ID'].isin(trans)]

#create pre_gene_list as list for swarm job later
pre_gene_list = coord['ID'].tolist()
len(pre_gene_list)

19083

In [58]:
#count how many MT are in coord
mt_coord = coord[coord['X.Chr'].str.contains('MT')]
print("Number of MT chromosom in coordinate data:" , mt_coord['X.Chr'].count())

#count how many X are in coord
y_coord = coord[coord['X.Chr'].str.contains('Y')]
print("Number of Y chromosom in coordinate data:" , y_coord['X.Chr'].count())

#count how many X are in coord
x_coord = coord[coord['X.Chr'].str.contains('X')]
print("Number of X chromosom in coordinate data:" , x_coord['X.Chr'].count())

#count how many X are in coord
gl_coord = coord[coord['X.Chr'].str.contains('GL')]
print("Number of GL chromosom in coordinate data:" , gl_coord['X.Chr'].count())

Number of MT chromosom in coordinate data: 13
Number of Y chromosom in coordinate data: 46
Number of X chromosom in coordinate data: 783
Number of GL chromosom in coordinate data: 0


In [65]:
#drop all the rows contain MT, X, Y, GL in X.Chr 
word_list = ["MT", "X", "Y", "GL"]
coord_df = coord[~coord['X.Chr'].str.contains('|'.join(word_list))]
coord_df = coord_df.reset_index(drop=True)
print("shape of coord_df:", coord_df.shape)
print(coord_df.head())

#create gene_list as list to drop columns in exp_case 
gene_list = coord_df['ID']
print("number of genes in gene_list:", len(gene_list))

#create gene_list as list to drop columns in exp_case 
gene_list_1 = coord_df['ID'].tolist()

#check miscoordinate numbers are same 602
no_coord = mis_coord['ID'].tolist()
print("number of missing genes:", len(no_coord))

shape of coord_df: (18241, 4)
  X.Chr   start     end               ID
0     1   69091   70008  ENSG00000186092
1     1  860260  879955  ENSG00000187634
2     1  879584  894689  ENSG00000188976
3     1  895967  901095  ENSG00000187961
4     1  901877  911245  ENSG00000187583
number of genes in gene_list: 18241
number of missing genes: 602


In [67]:
#save gene_list as txt -- run only once for download
gene_list.to_csv(r'./data_folder/gene_list.txt', sep='\t' ,index=False)

#save coord_df as txt -- run only once for download
coord_df.to_csv(r'./data_folder/twas_coordinate.txt', sep='\t' ,index=False)

In [61]:
#drop columns if not in gene_list_1
final_pheno = expression_case.drop(columns=[col for col in expression_case[expression_case.columns[2:19687]] if col not in gene_list_1])
final_pheno

,FID,IID,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000273088,ENSG00000273154,ENSG00000273155,ENSG00000273171,ENSG00000273173,ENSG00000273213,ENSG00000273217,ENSG00000273238,ENSG00000273274,ENSG00000273291
0,PD-PDAA503EF5,PD-PDAA503EF5,0.153490,-1.195026,-0.094739,0.278269,-1.606817,-1.231579,-0.471478,0.480406,...,-0.499505,0.519157,-0.078847,-0.499505,-0.265569,0.662116,-0.321552,-0.492056,-0.895260,0.836818
1,PD-PDAB074CYQ,PD-PDAB074CYQ,0.741327,0.725518,1.231583,0.593704,-1.066773,0.544353,-1.111941,0.219180,...,-0.499505,-1.235525,-0.291322,-0.499505,0.481206,0.671036,-0.282876,-0.326137,-0.624609,-0.643441
2,PD-PDAB549YWB,PD-PDAB549YWB,0.427937,-1.107550,0.040538,-1.836332,-0.428239,1.349854,-1.249725,-0.582143,...,-0.499505,0.628770,-0.542193,-0.499505,0.347083,-1.752264,-0.178388,-0.615766,0.517863,0.938400
3,PD-PDAB762PA3,PD-PDAB762PA3,-0.632429,-1.268311,0.100997,0.499943,-0.195359,1.295541,0.593312,-1.838987,...,-0.499505,0.833768,0.860871,-0.499505,-0.738382,0.277839,-0.103466,-0.259155,-0.890719,-0.630759
4,PD-PDAD926WRZ,PD-PDAD926WRZ,-1.023469,-2.544938,-0.100176,0.210245,0.317189,-0.789634,-0.752465,0.179003,...,0.037965,0.657849,0.597915,0.037965,0.003539,-0.053181,-0.095052,-0.571399,-1.044443,1.465210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,PP-4125,PP-4125,0.313107,0.175639,-0.246924,-0.490545,-2.050227,-0.381930,0.182633,0.045561,...,0.575435,-0.300371,-0.950092,0.575435,0.446657,-1.066448,-0.236436,-2.419127,-1.431688,-2.650735
1134,PP-4126,PP-4126,-0.509980,-1.396533,-0.740332,0.691257,0.945632,-1.188982,1.400997,1.025471,...,0.037965,1.070220,0.152859,0.037965,1.173718,-0.667721,-0.335580,0.467387,0.724382,0.816589
1135,PP-4127,PP-4127,-0.942894,1.153752,0.699754,0.230091,-0.138720,0.958663,-0.684057,-0.556005,...,-1.036976,-0.830943,-0.450324,-1.036976,0.675392,0.377462,-0.249957,-0.336044,0.367930,-0.612752
1136,PP-4135,PP-4135,0.334858,0.571822,-0.503539,-2.049354,3.221209,-0.065807,5.463260,1.003958,...,-1.036976,0.635543,-1.422952,-1.036976,0.783682,-0.779745,-0.131228,1.435043,0.387422,-0.793902


In [62]:
#save expression matrix as txt --run only once
final_pheno.to_csv(r'./data_folder/expression_matrix_final.txt', sep= '\t', index=False)

plink files (genomic data), expression data(phenomic data), coordinate data, covariate data, and gene list are ready to use

geno_path = f'{datadir}/qc_genotypes_twas'
#plink files to use: {datadir}/qc_genotypes_twas_hg19_lifted
gene_list_path = f'{datadir}/gene_list.txt'
pheno_path = f'{datadir}/expression_matrix_final.txt'
coord_path = f'{datadir}/twas_coordinate.txt'
covar_path = f'{datadir}/covariates.txt'

GWAS summery stat will be preprocessed after 1_liftover process

In [29]:
#open bim file 
bim_df = pd.read_csv("{datadir}/qc_genotypes_twas_hg19_lifted.bim", sep='\t', header=None)
bim_df.columns = ['Chr', 'Variant_ID', 'Position', 'Basepair coordinate', 'Allele_1', 'Allele_2']
bim_df.head()

,Chr,Variant_ID,Position,Basepair coordinate,Allele_1,Allele_2
0,1,rs145427775,0,10291,T,C
1,1,rs55998931,0,10492,T,C
2,1,rs199896944,0,13504,A,G
3,1,rs199856693,0,14933,A,G
4,1,rs201855936,0,14948,A,G


In [2]:
#check sumstat.csv
meta = pd.read_csv("sumstat.csv", sep=' ')
meta.head()

,MarkerName,Allele1,Allele2,Freq1,FreqSE,MinFreq,MaxFreq,Effect,StdErr,P-value
0,chr11:88249377,t,c,0.9905,0.0026,0.9882,0.9971,0.0238,0.0594,0.6889
1,chr1:60320992,a,g,0.9375,0.0049,0.9221,0.9473,0.0185,0.0210,0.3801
2,chr8:135908647,a,g,0.2112,0.0070,0.1703,0.2316,-0.0054,0.0123,0.6598
3,chr16:77148858,a,g,0.9963,0.0005,0.9961,0.9984,-0.0228,0.1356,0.8663
4,chr11:97895884,c,g,0.0587,0.0035,0.0361,0.0685,-0.0219,0.0217,0.3148
